In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [115]:
path = '/home/jupyter-l.fedoseeva-12/Lesson_5/data.csv.zip'

In [ ]:
#   6.1

In [152]:
retail = pd.read_csv(path, compression='zip', encoding='ISO-8859-1')

In [153]:
retail_columns = retail.columns

In [154]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [155]:
nrows = retail.shape[0]

In [156]:
nrows

541909

6.2   Проверьте, встречаются ли в данных повторяющиеся наблюдения, 
       и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.

In [157]:
retail_wo_dupl = retail.drop_duplicates()

In [158]:
nrows_wo_dupl = retail_wo_dupl.shape[0]

In [159]:
nrows_wo_dupl

536641

In [160]:
nrows - nrows_wo_dupl

5268

In [161]:
retail_wo_dupl.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


6.3  Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных. 
     Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 

In [55]:
retail_wo_dupl.InvoiceNo.str.startswith('C').sum()

9251

6.3   Теперь отфильтруйте данные и
      оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [162]:
retail_wo_dupl_0 = retail_wo_dupl.query('Quantity > 0')

6.4   Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). 
      Оставьте только тех, кто совершил более N транзакций (InvoiceNo), 
      где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).

In [163]:
retail_wo_Germ = retail_wo_dupl_0.query("Country == 'Germany'") \
                                .groupby('CustomerID', as_index=False) \
                                .agg({'InvoiceNo': 'nunique'}) \
                                .rename(columns={'InvoiceNo': 'Number_orders'}) \
                                .sort_values('Number_orders', ascending=False)

In [164]:
retail_wo_Germ.head()

,CustomerID,Number_orders
37,12569.0,32
3,12471.0,30
83,12720.0,25
52,12621.0,20
6,12474.0,18


In [165]:
quantile = retail_wo_Germ.Number_orders.quantile(q=0.8)

In [166]:
quantile

7.0

In [169]:
germany_top = retail_wo_Germ.query('Number_orders > {}'.format(quantile))['CustomerID'].reset_index(drop=True)

In [170]:
germany_top

0     12569.0
1     12471.0
2     12720.0
3     12621.0
4     12474.0
5     12709.0
6     12712.0
7     12647.0
8     12500.0
9     12662.0
10    12476.0
11    12481.0
12    12600.0
13    12708.0
14    12626.0
15    12705.0
16    12619.0
17    12524.0
Name: CustomerID, dtype: float64

6.5   Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и  
      оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в     
      top_retail_germany.

In [173]:
top_retail_germany = retail[retail.CustomerID.isin(germany_top)]

In [174]:
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


6.6    Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего,          кроме POST? 
       Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [176]:
top_retail_germany.groupby('StockCode') \
                    .agg({'InvoiceNo': 'count'}) \
                    .sort_values('InvoiceNo', ascending=False)

,InvoiceNo
StockCode,
POST,220
22326,55
22423,47
22328,41
20719,31
...,...
22569,1
22574,1
22576,1


6.7   Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity       и UnitPrice.

In [177]:
retail.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [192]:
retail_2 = retail_wo_dupl_0.assign(Revenue = retail.Quantity * retail.UnitPrice)

In [193]:
retail_2

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


6.8.    Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [222]:
retail_sum = retail_2.groupby('InvoiceNo',as_index=False) \
                    .agg({'Revenue': 'sum'}) \
                    .rename(columns={'Revenue': 'Total_revenue'}) \
                    .sort_values('Total_revenue', ascending=False) \
                    .nlargest(5,'Total_revenue')

In [223]:
retail_sum

,InvoiceNo,Total_revenue
20689,581483,168469.60
2202,541431,77183.60
17582,574941,52940.94
18251,576365,50653.91
9034,556444,38970.00


In [224]:
retail_number = retail_sum.InvoiceNo.values

In [225]:
retail_number

array(['581483', '541431', '574941', '576365', '556444'], dtype=object)